In [5]:
%load_ext autoreload
%autoreload 2
import torch
from argparse import ArgumentParser
import pytorch_lightning as pl
from project.data.data import *
from project.models.models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocess raw data:

In [2]:
#processor = RaceDataProcessor()
#processor.process_data("RACE", "LON")

## Train model

In [13]:
data_module.tokenizer.additional_special_tokens

['[CON]', '[QUE]', '[ANS]', '[DIS]']

In [11]:
# Parse arguments:
parser = ArgumentParser()
parser = RaceDataModule.add_model_specific_args(parser)
parser = RaceModule.add_model_specific_args(parser)
parser = pl.Trainer.add_argparse_args(parser)
args = parser.parse_args(
    "--data_path LON \
    --batch_size 8 \
    --num_workers 6 \
    --d_model 768 \
    --nhead 8 \
    --num_layers 1 \
    --learning_rate 1e-5 \
    --special_tokens [CON] [QUE] [ANS] [DIS] \
    --dm_pretrained_model distilbert-base-cased \
    --m_pretrained_model distilbert-base-cased \
    --gpus 1 \
    --max_epochs 5 \
    --check_val_every_n_epoch 1".split()
)

# Module and data module:
data_module = RaceDataModule(args)
module = RaceModule(args)

# Callbacks:
checkpoint = ModelCheckpoint(
    dirpath="./checkpoint/fx-{epoch:02d}-{val_loss:.7f}",
    monitor="val_loss"
)

# Trainer:
trainer = pl.Trainer.from_argparse_args(
    args,
    checkpoint_callback=checkpoint
)

trainer.fit(module, data_module)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name          | Type                    | Params
----------------------------------------------------------
0 | encoder       | DistilBertModel         | 65.2 M
1 | embedding     | Embeddings              | 22.7 M
2 | decoder_layer | TransformerDecoderLayer | 7.9 M 
3 | decoder       | TransformerDecoder      | 7.9 M 
4 | head          | Linear                  | 22.3 M
----------------------------------------------------------
38.1 M    Trainable params
65.2 M    Non-trainable params
103 M     Total params
413.009   Total estimated model params size (MB)


/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1